# Testing the model on new data

## Import Libraries

In [1]:
library('org.Hs.eg.db')
library(caret)
library(GeneAnswers)

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welco

## Download the data

[Amato et al, 2020](https://www.mdpi.com/2072-6694/12/7/1943/htm)<br>
[Suplementary Data](https://www.mdpi.com/2072-6694/12/7/1943/s1)<br>
[GEO Dataset](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE145996)<br>
[fpkm matrix](https://ftp.ncbi.nlm.nih.gov/geo/series/GSE145nnn/GSE145996/suppl/GSE145996_Melanoma_Immune_FPKM.txt.gz)<br>
[Data Guidelines](https://academic.oup.com/jnci/article/92/3/205/2965042)

In [2]:
system("wget --content-disposition https://ftp.ncbi.nlm.nih.gov/geo/series/GSE145nnn/GSE145996/suppl/GSE145996_Melanoma_Immune_FPKM.txt.gz")
system("gunzip GSE145996_Melanoma_Immune_FPKM.txt.gz")

## Read the data

In [19]:
expression <- t(read.csv("GSE145996_Melanoma_Immune_FPKM.txt", check.names=FALSE, sep="	"))
colnames(expression) <- expression[1,]
expression <- expression[3:nrow(expression),]
dim(expression)
expression

[1]    14 20115

,CD274,CD83,"MIR4647,NFKBIE,SLC35B2",NFKB1,NFKB2,PPP6C,"PPP6R1,TMEM86B",PPP6R2,RELA,"HLA-B,HLA-C",⋯,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Sample_01_MB_1692,1.70986e+00,1.40795e+01,8.26896e+01,6.86929e+00,5.40256e+01,3.93078e+00,4.54053e+01,1.90621e+01,2.42328e+01,3.46855e+03,⋯,3.19902e+00,1.15330e+01,2.15398e-01,3.62708e-01,1.53010e+01,0.00000e+00,1.30244e+00,8.34690e+01,1.70370e+01,2.80840e+00
Sample_02_MB_2674,9.26617e-01,1.43558e+01,1.10078e+02,7.18699e+00,2.13601e+01,9.48971e+00,3.32080e+01,2.32628e+01,3.72764e+01,3.64884e+03,⋯,6.32825e+00,1.20277e+01,6.69960e-01,1.66920e+00,9.69400e+00,1.79567e-02,3.16809e+00,2.13506e+02,7.70055e+00,8.85843e+00
Sample_03_MB_4103,1.40566e-01,3.78687e+00,1.88939e+02,4.46024e+00,1.59364e+01,5.96592e+00,3.14053e+01,1.52816e+01,2.93945e+01,5.51884e+02,⋯,3.58282e+00,1.25850e+01,5.81401e-01,1.30054e+00,5.65156e+00,0.00000e+00,2.52052e+00,5.15735e+01,5.39559e+00,8.88363e+00
Sample_04_MB_2750,4.51264e-01,6.85738e+00,9.68727e+01,8.48591e+00,1.04931e+01,8.88702e+00,2.36175e+01,9.97830e+00,5.07890e+01,2.77366e+02,⋯,4.69078e+00,3.08020e+01,4.68166e-01,1.18203e+00,6.98154e+00,0.00000e+00,2.36231e+00,3.90655e+01,1.10496e+01,8.30021e+00
Sample_05_MB_2781,1.43912e-01,5.66215e+00,1.07308e+02,8.53134e+00,8.14618e+00,1.47178e+01,5.22661e+01,1.66929e+01,3.77961e+01,6.17023e+02,⋯,4.89644e+00,1.39279e+01,4.04877e-01,1.24570e+00,7.60890e+00,0.00000e+00,2.04878e+00,7.47339e+01,1.11223e+01,6.37146e+00
Sample_06_MB_1882,3.04719e-01,3.36278e+01,2.02360e+02,5.82250e+00,1.71778e+01,1.15120e+01,2.62219e+01,2.81147e+01,5.08007e+01,1.37864e+03,⋯,4.92425e+00,2.38592e+01,2.66253e-01,9.47566e-01,1.03837e+01,0.00000e+00,1.84467e+00,8.65665e+01,1.47008e+01,4.89474e+00
Sample_07_MB_3025,4.33797e+00,4.32775e+01,3.59592e+01,2.73384e+01,3.33706e+01,2.50883e+01,4.93569e+01,1.57431e+01,3.18713e+01,1.52481e+03,⋯,1.49796e+01,3.01808e+01,8.86820e-01,2.45053e+00,7.60845e+00,4.59389e-01,2.83801e+00,5.84159e+01,8.88562e+00,1.12049e+01
Sample_08_MB_3178,6.69988e-01,1.33586e+01,1.02870e+02,1.11796e+01,1.55228e+01,7.04854e+00,4.57725e+01,2.16686e+01,2.07442e+01,1.35505e+03,⋯,3.25084e+00,1.37403e+01,5.46730e-01,1.05910e+00,8.79789e+00,0.00000e+00,2.46056e+00,1.06278e+02,9.53387e+00,5.62300e+00
Sample_09_MB_3838,3.28910e+00,8.49617e-01,1.19056e+02,3.59974e+00,1.36191e+01,2.74738e+00,3.01298e+01,3.48201e+01,4.38820e+01,2.57232e+03,⋯,1.07657e+00,3.70015e+00,3.03673e-01,5.72022e-01,1.24590e+01,2.51401e-01,1.82435e+00,6.61014e+01,1.05082e+01,4.29746e+00
Sample_10_MB_2786,1.97310e-01,5.52707e+00,1.17381e+02,7.64725e+00,9.42208e+00,1.21668e+01,5.83784e+01,1.82827e+01,3.85948e+01,7.21320e+02,⋯,4.20328e+00,1.24458e+01,3.69725e-01,1.27491e+00,7.64801e+00,0.00000e+00,1.91992e+00,7.59998e+01,1.03328e+01,5.98021e+00


### Format the data
#### Separated Columns
Some columns have gene expression for multiple genes<br>
Breaking those genes into separate columns

In [20]:
expr <- expression
for (i in colnames(expr)) {
    split_obj <- strsplit(i, ",")[[1]]
    if(length(split_obj) > 1){
#         print(split_obj)
        for (j in split_obj){
            expr <- cbind(expr, tmp=expr[,i])
            colnames(expr)[ncol(expr)] <- j
        }
    }
    
}
# removing all columns with multiple genes
remove <- grep(",", colnames(expr))
expr <- expr[, -remove]
row_names <- row.names(expr)
expr <- apply(expr, 2, as.numeric)
row.names(expr) <- row_names 
head(expr)

,CD274,CD83,NFKB1,NFKB2,PPP6C,PPP6R2,RELA,HLA-C,HLA-DQB2,HLA-DPB1,⋯,ZNF621,ZNF670,ZNF670-ZNF695,ZNF695,ZNF69,ZNF700,ZNF746,ZNF767P,ZNF780A,ZNF780B
Sample_01_MB_1692,1.709860,14.07950,6.86929,54.02560,3.93078,19.0621,24.2328,0.0000,13.128000,37.06150,⋯,9.908190,1.44021,1.44021,1.44021,10.02540,10.02540,13.89420,13.89420,2.98094,2.98094
Sample_02_MB_2674,0.926617,14.35580,7.18699,21.36010,9.48971,23.2628,37.2764,0.0000,2.000370,11.80040,⋯,3.384830,3.01956,3.01956,3.01956,9.98321,9.98321,9.43907,9.43907,2.31031,2.31031
Sample_03_MB_4103,0.140566,3.78687,4.46024,15.93640,5.96592,15.2816,29.3945,88.9503,2.705590,10.07920,⋯,0.493359,1.26823,1.26823,1.26823,2.97304,2.97304,9.05069,9.05069,2.87643,2.87643
Sample_04_MB_2750,0.451264,6.85738,8.48591,10.49310,8.88702,9.9783,50.7890,382.8110,0.178607,8.69620,⋯,4.101550,2.85478,2.85478,2.85478,7.30161,7.30161,5.14282,5.14282,1.78122,1.78122
Sample_05_MB_2781,0.143912,5.66215,8.53134,8.14618,14.71780,16.6929,37.7961,0.0000,1.502180,8.80717,⋯,4.059610,1.51632,1.51632,1.51632,5.78163,5.78163,13.46300,13.46300,3.03626,3.03626
Sample_06_MB_1882,0.304719,33.62780,5.82250,17.17780,11.51200,28.1147,50.8007,352.7650,1.838130,34.20850,⋯,2.881340,1.39493,1.39493,1.39493,4.27304,4.27304,7.97705,7.97705,2.20541,2.20541


#### Map Gene Names to Entrez IDs

In [21]:
# mapping gene names to Entrez IDs
colnames(expr) <- mapIds(org.Hs.eg.db, colnames(expr), 'ENTREZID', 'ALIAS')

'select()' returned 1:many mapping between keys and columns



#### Remove Duplicates

In [22]:
# Remove duplicate columns
res <- list()
uniqCname <- unique(colnames(expr))
for (i in 1:length(uniqCname)){
    Cname <- uniqCname[i]
    count <- length(which(colnames(expr) %in% Cname))
    pos <- which(colnames(expr) %in% Cname)
    if(count > 1){
        tmp <- expr[, pos]
        tmp.mean <- colMeans(tmp)
        tmp.mean.max <- max(tmp.mean)
        final_pos <- which(tmp.mean %in% tmp.mean.max)[1]
        tmp <- tmp[, final_pos]
    }else{
        tmp <- expr[,pos]
    }
    res[[i]] <- tmp
    names(res)[i] <- Cname 
}

expr_unique <- do.call(cbind, res)


# Converting to log2(x)

expr_unique[expr_unique == 0] <- 0.001

# convert values to log
expr_unique <- log(expr_unique, 2)

In [15]:
# mapIds(org.Hs.eg.db, colnames(expr), 'ENTREZID', 'SYMBOL')

## Clinical Data

In [23]:
clinical <- data.frame(row.names=c("Sample_01_MB_1692", "Sample_02_MB_2674", "Sample_03_MB_4103", "Sample_04_MB_2750",
                                 "Sample_05_MB_2781", "Sample_06_MB_1882", "Sample_07_MB_3025", "Sample_08_MB_3178",
                                 "Sample_09_MB_3838", "Sample_10_MB_2786", "Sample_11_MB_2855", "Sample_12_MB_3316",
                                 "Sample_13_MB_3432", "Sample_14_MB_4117"), 
                       Age=c(23, 79, 41, 57, 59, 88, 49, 78, 37, 41, 22, 56, 57, 73),
                       Gender=c("F", "F", "M", "M", "F", "M", "F", "M", "M", "M", "F", "F", "M", "M"), 
                       ResponseClass=c("CR", "CR", "PR", "PR", "PR", "PR", "PR", "PR", "SD", "PD", "PD", "PD", "PD", "PD"),
                       Response=c("Y", "Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "N", "N", "N", "N", "N")
                      )
clinical$Response <- as.factor(clinical$Response)
clinical

,Age,Gender,ResponseClass,Response
,<dbl>,<chr>,<chr>,<fct>
Sample_01_MB_1692,23,F,CR,Y
Sample_02_MB_2674,79,F,CR,Y
Sample_03_MB_4103,41,M,PR,Y
Sample_04_MB_2750,57,M,PR,Y
Sample_05_MB_2781,59,F,PR,Y
Sample_06_MB_1882,88,M,PR,Y
Sample_07_MB_3025,49,F,PR,Y
Sample_08_MB_3178,78,M,PR,Y
Sample_09_MB_3838,37,M,SD,N


## Merge Clinical and Expression Data

In [24]:
amato_combined <- merge(clinical,expr_unique,by='row.names',all=TRUE)
colnames(amato_combined)[colnames(amato_combined) == "Row.names"] <- "Patient"
amato_combined

Patient,Age,Gender,ResponseClass,Response,29126,9308,4790,4791,5537,⋯,285268,93474,100533111,57116,7620,90592,155061,79970,284323,163131
<I<chr>>,<dbl>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Sample_01_MB_1692,23,F,CR,Y,0.7738782,3.8155242,2.780161,5.755571,1.974816,⋯,3.3086215,0.5262792,0.5262792,0.5262792,3.325588,3.325588,3.796411,3.796411,1.5757673,1.5757673
Sample_02_MB_2674,79,F,CR,Y,-0.1099549,3.8435618,2.845388,4.416846,3.246364,⋯,1.7590834,1.5943383,1.5943383,1.5943383,3.319504,3.319504,3.238645,3.238645,1.2080864,1.2080864
Sample_03_MB_4103,41,M,PR,Y,-2.8306804,1.9210059,2.157121,3.994254,2.576745,⋯,-1.0192903,0.3428164,0.3428164,0.3428164,1.571939,1.571939,3.178028,3.178028,1.5242794,1.5242794
Sample_04_MB_2750,57,M,PR,Y,-1.1479564,2.7776575,3.085069,3.391369,3.151700,⋯,2.0361692,1.5133796,1.5133796,1.5133796,2.868215,2.868215,2.362560,2.362560,0.8328657,0.8328657
Sample_05_MB_2781,59,F,PR,Y,-2.7967412,2.5013500,3.092772,3.026124,3.879490,⋯,2.0213411,0.6005742,0.6005742,0.6005742,2.531476,2.531476,3.750928,3.750928,1.6022953,1.6022953
Sample_06_MB_1882,88,M,PR,Y,-1.7144486,5.0715825,2.541639,4.102473,3.525067,⋯,1.5267399,0.4801927,0.4801927,0.4801927,2.095263,2.095263,2.995855,2.995855,1.1410469,1.1410469
Sample_07_MB_3025,49,F,PR,Y,2.1170201,5.4355453,4.772857,5.060506,4.648943,⋯,1.6772511,0.9112621,0.9112621,0.9112621,3.321006,3.321006,2.911839,2.911839,2.2826207,2.2826207
Sample_08_MB_3178,78,M,PR,Y,-0.5777928,3.7396969,3.482797,3.956317,2.817324,⋯,1.8862146,-0.5967214,-0.5967214,-0.5967214,2.317596,2.317596,3.179627,3.179627,1.2044225,1.2044225
Sample_09_MB_3838,37,M,SD,N,1.7176929,-0.2351155,1.847893,3.767559,1.458056,⋯,4.1004311,-2.0114135,-2.0114135,-2.0114135,4.363704,4.363704,4.134788,4.134788,2.3697429,2.3697429


In [25]:
write.csv(amato_combined, "/home/jp/ICP_Responders/DataCollectionFormatting/MergeTables/Amato.csv", row.names = FALSE)

# Modelling

## GBM

### MergedRank200

In [9]:
gbm_mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_mergedRank.rds")
feature.mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.mergedRank.rds")

res <- list()
tmp <- feature.mergedRank[, -which(colnames(feature.mergedRank) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_mergedRank <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_mergedRank) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_mergedRank[gbm_inp_mergedRank == 0] <- 0.001

# # convert values to log
# gbm_inp_mergedRank <- log(gbm_inp_mergedRank, 2)

gbm_predictions_mergedRank <- predict(gbm_mergedRank, gbm_inp_mergedRank)
confusionMatrix(data = gbm_predictions_mergedRank, reference = clinical$Response, positive = "Y")

[1]  2 45

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 0
         Y 6 8
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.61052         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.5714          
         Neg Pred Value :    NaN          
             Prevalence : 0.5714          
         Detection Rate : 0.5714          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : Y               
                                          

### Variance

In [10]:
gbm_top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_top200var.rds")
feature.top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.top200var.rds")

res <- list()
tmp <- feature.top200var[, -which(colnames(feature.top200var) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_top200var <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_top200var) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_top200var[gbm_inp_top200var == 0] <- 0.001

# # convert values to log
# gbm_inp_top200var <- log(gbm_inp_top200var, 2)

gbm_predictions_top200var <- predict(gbm_top200var, gbm_inp_top200var)
confusionMatrix(data = gbm_predictions_top200var, reference = clinical$Response, positive = "Y")

[1]  33  78 144 150 163 164 165 166 168 169 173 179 195

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 0
         Y 6 8
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.61052         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.5714          
         Neg Pred Value :    NaN          
             Prevalence : 0.5714          
         Detection Rate : 0.5714          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : Y               
                                          

### FCBF

In [11]:
gbm_fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_fcbf.rds")
feature.fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.fcbf.rds")

res <- list()
tmp <- feature.fcbf[, -which(colnames(feature.fcbf) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_fcbf <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_fcbf) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_fcbf[gbm_inp_fcbf == 0] <- 0.001

# # convert values to log
# gbm_inp_fcbf <- log(gbm_inp_fcbf, 2)

gbm_predictions_fcbf <- predict(gbm_fcbf, gbm_inp_fcbf)
confusionMatrix(data = gbm_predictions_fcbf, reference = clinical$Response, positive = "Y")

[1] 17 18 29

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 5 4
         Y 1 4
                                          
               Accuracy : 0.6429          
                 95% CI : (0.3514, 0.8724)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.3990          
                                          
                  Kappa : 0.3137          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.5000          
            Specificity : 0.8333          
         Pos Pred Value : 0.8000          
         Neg Pred Value : 0.5556          
             Prevalence : 0.5714          
         Detection Rate : 0.2857          
   Detection Prevalence : 0.3571          
      Balanced Accuracy : 0.6667          
                                          
       'Positive' Class : Y               
                                          

### Limma

In [12]:
gbm_limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_limma.rds")
feature.limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.limma.rds")

res <- list()
tmp <- feature.limma[, -which(colnames(feature.limma) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_limma <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_limma) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_limma[gbm_inp_limma == 0] <- 0.001

# # convert values to log
# gbm_inp_limma <- log(gbm_inp_limma, 2)

gbm_predictions_limma <- predict(gbm_limma, gbm_inp_limma)
confusionMatrix(data = gbm_predictions_limma, reference = clinical$Response, positive = "Y")

[1] 50

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 4 6
         Y 2 2
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9108          
                                          
                  Kappa : -0.0769         
                                          
 Mcnemar's Test P-Value : 0.2888          
                                          
            Sensitivity : 0.2500          
            Specificity : 0.6667          
         Pos Pred Value : 0.5000          
         Neg Pred Value : 0.4000          
             Prevalence : 0.5714          
         Detection Rate : 0.1429          
   Detection Prevalence : 0.2857          
      Balanced Accuracy : 0.4583          
                                          
       'Positive' Class : Y               
                                          

### Random Forest Permutation

In [13]:
gbm_ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_ranger.rds")
feature.ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.ranger.rds")

res <- list()
tmp <- feature.ranger[, -which(colnames(feature.ranger) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_ranger <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_ranger) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_ranger[gbm_inp_ranger == 0] <- 0.001

# # convert values to log
# gbm_inp_ranger <- log(gbm_inp_ranger, 2)

gbm_predictions_ranger <- predict(gbm_ranger, gbm_inp_ranger)
confusionMatrix(data = gbm_predictions_ranger, reference = clinical$Response, positive = "Y")

[1]  35  54  70  87 126 174 200

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 2 1
         Y 4 7
                                          
               Accuracy : 0.6429          
                 95% CI : (0.3514, 0.8724)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.3990          
                                          
                  Kappa : 0.2222          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.3333          
         Pos Pred Value : 0.6364          
         Neg Pred Value : 0.6667          
             Prevalence : 0.5714          
         Detection Rate : 0.5000          
   Detection Prevalence : 0.7857          
      Balanced Accuracy : 0.6042          
                                          
       'Positive' Class : Y               
                                          

### Mutual Information

In [14]:
gbm_jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_jmim.rds")
feature.jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.jmim.rds")

res <- list()
tmp <- feature.jmim[, -which(colnames(feature.jmim) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_jmim <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_jmim) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_jmim[gbm_inp_jmim == 0] <- 0.001

# # convert values to log
# gbm_inp_jmim <- log(gbm_inp_jmim, 2)

gbm_predictions_jmim <- predict(gbm_jmim, gbm_inp_jmim)
confusionMatrix(data = gbm_predictions_jmim, reference = clinical$Response, positive = "Y")

[1]  25  36  84 114 144 154

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 3 5
         Y 3 3
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9108          
                                          
                  Kappa : -0.12           
                                          
 Mcnemar's Test P-Value : 0.7237          
                                          
            Sensitivity : 0.3750          
            Specificity : 0.5000          
         Pos Pred Value : 0.5000          
         Neg Pred Value : 0.3750          
             Prevalence : 0.5714          
         Detection Rate : 0.2143          
   Detection Prevalence : 0.4286          
      Balanced Accuracy : 0.4375          
                                          
       'Positive' Class : Y               
                                          

### Literature Survey

In [15]:
gbm_litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_litSur.rds")
feature.litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.litSur.rds")

res <- list()
tmp <- feature.litSur[, -which(colnames(feature.litSur) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_litSur <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_litSur) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_litSur[gbm_inp_litSur == 0] <- 0.001

# # convert values to log
# gbm_inp_litSur <- log(gbm_inp_litSur, 2)

gbm_predictions_litSur <- predict(gbm_litSur, gbm_inp_litSur)
confusionMatrix(data = gbm_predictions_litSur, reference = clinical$Response, positive = "Y")

integer(0)

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 0
         Y 6 8
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.61052         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.5714          
         Neg Pred Value :    NaN          
             Prevalence : 0.5714          
         Detection Rate : 0.5714          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : Y               
                                          

### Prat et al.

In [16]:
gbm_prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/gbm_prat.rds")
feature.prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.prat.rds")

res <- list()
tmp <- feature.prat[, -which(colnames(feature.prat) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

gbm_inp_prat <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(gbm_inp_prat) %in% 0.001)

# # Replace 0 with 0.001
# gbm_inp_prat[gbm_inp_prat == 0] <- 0.001

# # convert values to log
# gbm_inp_prat <- log(gbm_inp_prat, 2)

gbm_predictions_prat <- predict(gbm_prat, gbm_inp_prat)
confusionMatrix(data = gbm_predictions_prat, reference = clinical$Response, positive = "Y")

[1]  58 104 179 266 306 311 330 372 425 445 449

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 4 6
         Y 2 2
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9108          
                                          
                  Kappa : -0.0769         
                                          
 Mcnemar's Test P-Value : 0.2888          
                                          
            Sensitivity : 0.2500          
            Specificity : 0.6667          
         Pos Pred Value : 0.5000          
         Neg Pred Value : 0.4000          
             Prevalence : 0.5714          
         Detection Rate : 0.1429          
   Detection Prevalence : 0.2857          
      Balanced Accuracy : 0.4583          
                                          
       'Positive' Class : Y               
                                          

## Cforest

### MergedRank200

In [19]:
cforest_mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_mergedRank.rds")
feature.mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.mergedRank.rds")

res <- list()
tmp <- feature.mergedRank[, -which(colnames(feature.mergedRank) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_mergedRank <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_mergedRank) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_mergedRank[cforest_inp_mergedRank == 0] <- 0.001

# # convert values to log
# cforest_inp_mergedRank <- log(cforest_inp_mergedRank, 2)

cforest_predictions_mergedRank <- predict(cforest_mergedRank, cforest_inp_mergedRank)
confusionMatrix(data = cforest_predictions_mergedRank, reference = clinical$Response, positive = "Y")

[1]  2 45

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 5 8
         Y 1 0
                                          
               Accuracy : 0.3571          
                 95% CI : (0.1276, 0.6486)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9703          
                                          
                  Kappa : -0.1455         
                                          
 Mcnemar's Test P-Value : 0.0455          
                                          
            Sensitivity : 0.00000         
            Specificity : 0.83333         
         Pos Pred Value : 0.00000         
         Neg Pred Value : 0.38462         
             Prevalence : 0.57143         
         Detection Rate : 0.00000         
   Detection Prevalence : 0.07143         
      Balanced Accuracy : 0.41667         
                                          
       'Positive' Class : Y               
                                          

### Variance

In [22]:
cforest_top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_top200var.rds")
feature.top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.top200var.rds")

res <- list()
tmp <- feature.top200var[, -which(colnames(feature.top200var) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_top200var <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_top200var) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_top200var[cforest_inp_top200var == 0] <- 0.001

# # convert values to log
# cforest_inp_top200var <- log(cforest_inp_top200var, 2)

cforest_predictions_top200var <- predict(cforest_top200var, cforest_inp_top200var)
confusionMatrix(data = cforest_predictions_top200var, reference = clinical$Response, positive = "Y")

[1]  33  78 144 150 163 164 165 166 168 169 173 179 195

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 2
         Y 6 6
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9108          
                                          
                  Kappa : -0.2727         
                                          
 Mcnemar's Test P-Value : 0.2888          
                                          
            Sensitivity : 0.7500          
            Specificity : 0.0000          
         Pos Pred Value : 0.5000          
         Neg Pred Value : 0.0000          
             Prevalence : 0.5714          
         Detection Rate : 0.4286          
   Detection Prevalence : 0.8571          
      Balanced Accuracy : 0.3750          
                                          
       'Positive' Class : Y               
                                          

### FCBF

In [23]:
cforest_fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_fcbf.rds")
feature.fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.fcbf.rds")

res <- list()
tmp <- feature.fcbf[, -which(colnames(feature.fcbf) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_fcbf <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_fcbf) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_fcbf[cforest_inp_fcbf == 0] <- 0.001

# # convert values to log
# cforest_inp_fcbf <- log(cforest_inp_fcbf, 2)

cforest_predictions_fcbf <- predict(cforest_fcbf, cforest_inp_fcbf)
confusionMatrix(data = cforest_predictions_fcbf, reference = clinical$Response, positive = "Y")

[1] 17 18 29

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 5 5
         Y 1 3
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.6105          
                                          
                  Kappa : 0.1923          
                                          
 Mcnemar's Test P-Value : 0.2207          
                                          
            Sensitivity : 0.3750          
            Specificity : 0.8333          
         Pos Pred Value : 0.7500          
         Neg Pred Value : 0.5000          
             Prevalence : 0.5714          
         Detection Rate : 0.2143          
   Detection Prevalence : 0.2857          
      Balanced Accuracy : 0.6042          
                                          
       'Positive' Class : Y               
                                          

### Limma

In [24]:
cforest_limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_limma.rds")
feature.limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.limma.rds")

res <- list()
tmp <- feature.limma[, -which(colnames(feature.limma) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_limma <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_limma) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_limma[cforest_inp_limma == 0] <- 0.001

# # convert values to log
# cforest_inp_limma <- log(cforest_inp_limma, 2)

cforest_predictions_limma <- predict(cforest_limma, cforest_inp_limma)
confusionMatrix(data = cforest_predictions_limma, reference = clinical$Response, positive = "Y")

[1] 50

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 6 8
         Y 0 0
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.91083         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.01333         
                                          
            Sensitivity : 0.0000          
            Specificity : 1.0000          
         Pos Pred Value :    NaN          
         Neg Pred Value : 0.4286          
             Prevalence : 0.5714          
         Detection Rate : 0.0000          
   Detection Prevalence : 0.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : Y               
                                          

### Random Forest Permutation

In [25]:
cforest_ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_ranger.rds")
feature.ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.ranger.rds")

res <- list()
tmp <- feature.ranger[, -which(colnames(feature.ranger) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_ranger <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_ranger) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_ranger[cforest_inp_ranger == 0] <- 0.001

# # convert values to log
# cforest_inp_ranger <- log(cforest_inp_ranger, 2)

cforest_predictions_ranger <- predict(cforest_ranger, cforest_inp_ranger)
confusionMatrix(data = cforest_predictions_ranger, reference = clinical$Response, positive = "Y")

[1]  35  54  70  87 126 174 200

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 6 7
         Y 0 1
                                          
               Accuracy : 0.5             
                 95% CI : (0.2304, 0.7696)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.79184         
                                          
                  Kappa : 0.1091          
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 0.12500         
            Specificity : 1.00000         
         Pos Pred Value : 1.00000         
         Neg Pred Value : 0.46154         
             Prevalence : 0.57143         
         Detection Rate : 0.07143         
   Detection Prevalence : 0.07143         
      Balanced Accuracy : 0.56250         
                                          
       'Positive' Class : Y               
                                          

### Mutual Information

In [26]:
cforest_jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_jmim.rds")
feature.jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.jmim.rds")

res <- list()
tmp <- feature.jmim[, -which(colnames(feature.jmim) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_jmim <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_jmim) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_jmim[cforest_inp_jmim == 0] <- 0.001

# # convert values to log
# cforest_inp_jmim <- log(cforest_inp_jmim, 2)

cforest_predictions_jmim <- predict(cforest_jmim, cforest_inp_jmim)
confusionMatrix(data = cforest_predictions_jmim, reference = clinical$Response, positive = "Y")

[1]  25  36  84 114 144 154

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 6 6
         Y 0 2
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.61052         
                                          
                  Kappa : 0.2222          
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 0.2500          
            Specificity : 1.0000          
         Pos Pred Value : 1.0000          
         Neg Pred Value : 0.5000          
             Prevalence : 0.5714          
         Detection Rate : 0.1429          
   Detection Prevalence : 0.1429          
      Balanced Accuracy : 0.6250          
                                          
       'Positive' Class : Y               
                                          

### Literature Survey

In [27]:
cforest_litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_litSur.rds")
feature.litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.litSur.rds")

res <- list()
tmp <- feature.litSur[, -which(colnames(feature.litSur) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_litSur <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_litSur) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_litSur[cforest_inp_litSur == 0] <- 0.001

# # convert values to log
# cforest_inp_litSur <- log(cforest_inp_litSur, 2)

cforest_predictions_litSur <- predict(cforest_litSur, cforest_inp_litSur)
confusionMatrix(data = cforest_predictions_litSur, reference = clinical$Response, positive = "Y")

integer(0)

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 3 6
         Y 3 2
                                          
               Accuracy : 0.3571          
                 95% CI : (0.1276, 0.6486)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9703          
                                          
                  Kappa : -0.2353         
                                          
 Mcnemar's Test P-Value : 0.5050          
                                          
            Sensitivity : 0.2500          
            Specificity : 0.5000          
         Pos Pred Value : 0.4000          
         Neg Pred Value : 0.3333          
             Prevalence : 0.5714          
         Detection Rate : 0.1429          
   Detection Prevalence : 0.3571          
      Balanced Accuracy : 0.3750          
                                          
       'Positive' Class : Y               
                                          

### Prat et al.

In [28]:
cforest_prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/cforest_prat.rds")
feature.prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.prat.rds")

res <- list()
tmp <- feature.prat[, -which(colnames(feature.prat) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

cforest_inp_prat <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(cforest_inp_prat) %in% 0.001)

# # Replace 0 with 0.001
# cforest_inp_prat[cforest_inp_prat == 0] <- 0.001

# # convert values to log
# cforest_inp_prat <- log(cforest_inp_prat, 2)

cforest_predictions_prat <- predict(cforest_prat, cforest_inp_prat)
confusionMatrix(data = cforest_predictions_prat, reference = clinical$Response, positive = "Y")

[1]  58 104 179 266 306 311 330 372 425 445 449

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 3 3
         Y 3 5
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.6105          
                                          
                  Kappa : 0.125           
                                          
 Mcnemar's Test P-Value : 1.0000          
                                          
            Sensitivity : 0.6250          
            Specificity : 0.5000          
         Pos Pred Value : 0.6250          
         Neg Pred Value : 0.5000          
             Prevalence : 0.5714          
         Detection Rate : 0.3571          
   Detection Prevalence : 0.5714          
      Balanced Accuracy : 0.5625          
                                          
       'Positive' Class : Y               
                                          

# Naive Bayes

## MergedRank200

In [30]:
nb_mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_mergedRank.rds")
feature.mergedRank <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.mergedRank.rds")

res <- list()
tmp <- feature.mergedRank[, -which(colnames(feature.mergedRank) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_mergedRank <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_mergedRank) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_mergedRank[nb_inp_mergedRank == 0] <- 0.001

# # convert values to log
# nb_inp_mergedRank <- log(nb_inp_mergedRank, 2)

nb_predictions_mergedRank <- predict(nb_mergedRank, nb_inp_mergedRank)
confusionMatrix(data = nb_predictions_mergedRank, reference = clinical$Response, positive = "Y")

[1]  2 45

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 2 3
         Y 4 5
                                          
               Accuracy : 0.5             
                 95% CI : (0.2304, 0.7696)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.7918          
                                          
                  Kappa : -0.0426         
                                          
 Mcnemar's Test P-Value : 1.0000          
                                          
            Sensitivity : 0.6250          
            Specificity : 0.3333          
         Pos Pred Value : 0.5556          
         Neg Pred Value : 0.4000          
             Prevalence : 0.5714          
         Detection Rate : 0.3571          
   Detection Prevalence : 0.6429          
      Balanced Accuracy : 0.4792          
                                          
       'Positive' Class : Y               
                                          

## Variance

In [31]:
nb_top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_top200var.rds")
feature.top200var <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.top200var.rds")

res <- list()
tmp <- feature.top200var[, -which(colnames(feature.top200var) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_top200var <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_top200var) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_top200var[nb_inp_top200var == 0] <- 0.001

# # convert values to log
# nb_inp_top200var <- log(nb_inp_top200var, 2)

nb_predictions_top200var <- predict(nb_top200var, nb_inp_top200var)
confusionMatrix(data = nb_predictions_top200var, reference = clinical$Response, positive = "Y")

[1]  33  78 144 150 163 164 165 166 168 169 173 179 195

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 1 1
         Y 5 7
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.6105          
                                          
                  Kappa : 0.0455          
                                          
 Mcnemar's Test P-Value : 0.2207          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.1667          
         Pos Pred Value : 0.5833          
         Neg Pred Value : 0.5000          
             Prevalence : 0.5714          
         Detection Rate : 0.5000          
   Detection Prevalence : 0.8571          
      Balanced Accuracy : 0.5208          
                                          
       'Positive' Class : Y               
                                          

## FCBF

In [32]:
nb_fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_fcbf.rds")
feature.fcbf <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.fcbf.rds")

res <- list()
tmp <- feature.fcbf[, -which(colnames(feature.fcbf) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_fcbf <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_fcbf) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_fcbf[nb_inp_fcbf == 0] <- 0.001

# # convert values to log
# nb_inp_fcbf <- log(nb_inp_fcbf, 2)

nb_predictions_fcbf <- predict(nb_fcbf, nb_inp_fcbf)
confusionMatrix(data = nb_predictions_fcbf, reference = clinical$Response, positive = "Y")

[1] 17 18 29

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 3 2
         Y 3 6
                                          
               Accuracy : 0.6429          
                 95% CI : (0.3514, 0.8724)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.399           
                                          
                  Kappa : 0.2553          
                                          
 Mcnemar's Test P-Value : 1.000           
                                          
            Sensitivity : 0.7500          
            Specificity : 0.5000          
         Pos Pred Value : 0.6667          
         Neg Pred Value : 0.6000          
             Prevalence : 0.5714          
         Detection Rate : 0.4286          
   Detection Prevalence : 0.6429          
      Balanced Accuracy : 0.6250          
                                          
       'Positive' Class : Y               
                                          

## Limma

In [33]:
nb_limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_limma.rds")
feature.limma <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.limma.rds")

res <- list()
tmp <- feature.limma[, -which(colnames(feature.limma) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_limma <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_limma) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_limma[nb_inp_limma == 0] <- 0.001

# # convert values to log
# nb_inp_limma <- log(nb_inp_limma, 2)

nb_predictions_limma <- predict(nb_limma, nb_inp_limma)
confusionMatrix(data = nb_predictions_limma, reference = clinical$Response, positive = "Y")

[1] 50

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 4 5
         Y 2 3
                                          
               Accuracy : 0.5             
                 95% CI : (0.2304, 0.7696)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.7918          
                                          
                  Kappa : 0.0392          
                                          
 Mcnemar's Test P-Value : 0.4497          
                                          
            Sensitivity : 0.3750          
            Specificity : 0.6667          
         Pos Pred Value : 0.6000          
         Neg Pred Value : 0.4444          
             Prevalence : 0.5714          
         Detection Rate : 0.2143          
   Detection Prevalence : 0.3571          
      Balanced Accuracy : 0.5208          
                                          
       'Positive' Class : Y               
                                          

## Random Forest Permutation

In [34]:
nb_ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_ranger.rds")
feature.ranger <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.ranger.rds")

res <- list()
tmp <- feature.ranger[, -which(colnames(feature.ranger) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_ranger <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_ranger) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_ranger[nb_inp_ranger == 0] <- 0.001

# # convert values to log
# nb_inp_ranger <- log(nb_inp_ranger, 2)

nb_predictions_ranger <- predict(nb_ranger, nb_inp_ranger)
confusionMatrix(data = nb_predictions_ranger, reference = clinical$Response, positive = "Y")

[1]  35  54  70  87 126 174 200

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 2 2
         Y 4 6
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.6105          
                                          
                  Kappa : 0.087           
                                          
 Mcnemar's Test P-Value : 0.6831          
                                          
            Sensitivity : 0.7500          
            Specificity : 0.3333          
         Pos Pred Value : 0.6000          
         Neg Pred Value : 0.5000          
             Prevalence : 0.5714          
         Detection Rate : 0.4286          
   Detection Prevalence : 0.7143          
      Balanced Accuracy : 0.5417          
                                          
       'Positive' Class : Y               
                                          

## Mutual Information

In [35]:
nb_jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_jmim.rds")
feature.jmim <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.jmim.rds")

res <- list()
tmp <- feature.jmim[, -which(colnames(feature.jmim) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_jmim <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_jmim) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_jmim[nb_inp_jmim == 0] <- 0.001

# # convert values to log
# nb_inp_jmim <- log(nb_inp_jmim, 2)

nb_predictions_jmim <- predict(nb_jmim, nb_inp_jmim)
confusionMatrix(data = nb_predictions_jmim, reference = clinical$Response, positive = "Y")

[1]  25  36  84 114 144 154

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 0
         Y 6 8
                                          
               Accuracy : 0.5714          
                 95% CI : (0.2886, 0.8234)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.61052         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.04123         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.5714          
         Neg Pred Value :    NaN          
             Prevalence : 0.5714          
         Detection Rate : 0.5714          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : Y               
                                          

## Literature Survey

In [36]:
nb_litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_litSur.rds")
feature.litSur <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.litSur.rds")

res <- list()
tmp <- feature.litSur[, -which(colnames(feature.litSur) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_litSur <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_litSur) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_litSur[nb_inp_litSur == 0] <- 0.001

# # convert values to log
# nb_inp_litSur <- log(nb_inp_litSur, 2)

nb_predictions_litSur <- predict(nb_litSur, nb_inp_litSur)
confusionMatrix(data = nb_predictions_litSur, reference = clinical$Response, positive = "Y")

integer(0)

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 3 5
         Y 3 3
                                          
               Accuracy : 0.4286          
                 95% CI : (0.1766, 0.7114)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.9108          
                                          
                  Kappa : -0.12           
                                          
 Mcnemar's Test P-Value : 0.7237          
                                          
            Sensitivity : 0.3750          
            Specificity : 0.5000          
         Pos Pred Value : 0.5000          
         Neg Pred Value : 0.3750          
             Prevalence : 0.5714          
         Detection Rate : 0.2143          
   Detection Prevalence : 0.4286          
      Balanced Accuracy : 0.4375          
                                          
       'Positive' Class : Y               
                                          

## Prat et al.

In [37]:
nb_prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/nb_prat.rds")
feature.prat <- readRDS("/home/jp/ICP_Responders/ML/GideHugoRiaz_AntiPD1/Pre-processingFeatureSelection/Model testing/SavedModels/feature.prat.rds")

res <- list()
tmp <- feature.prat[, -which(colnames(feature.prat) %in% "Response")]

for (i in 1:ncol(tmp)){
    gene_name <- colnames(tmp)[i]
    check_gene <- length(which(colnames(expr_unique) %in% gene_name))
    if(check_gene == 1){
        res[[i]] <- expr_unique[,which(colnames(expr_unique) %in% gene_name)]
    }else{
        res[[i]] <- rep(0.001, nrow(expr_unique))
    }
    names(res)[i] <- gene_name 
}

nb_inp_prat <- do.call(cbind, res)

# which columns have missing genes
which(colMeans(nb_inp_prat) %in% 0.001)

# # Replace 0 with 0.001
# nb_inp_prat[nb_inp_prat == 0] <- 0.001

# # convert values to log
# nb_inp_prat <- log(nb_inp_prat, 2)

nb_predictions_prat <- predict(nb_prat, nb_inp_prat)
confusionMatrix(data = nb_predictions_prat, reference = clinical$Response, positive = "Y")

[1]  58 104 179 266 306 311 330 372 425 445 449

Confusion Matrix and Statistics

          Reference
Prediction N Y
         N 0 1
         Y 6 7
                                          
               Accuracy : 0.5             
                 95% CI : (0.2304, 0.7696)
    No Information Rate : 0.5714          
    P-Value [Acc > NIR] : 0.7918          
                                          
                  Kappa : -0.1395         
                                          
 Mcnemar's Test P-Value : 0.1306          
                                          
            Sensitivity : 0.8750          
            Specificity : 0.0000          
         Pos Pred Value : 0.5385          
         Neg Pred Value : 0.0000          
             Prevalence : 0.5714          
         Detection Rate : 0.5000          
   Detection Prevalence : 0.9286          
      Balanced Accuracy : 0.4375          
                                          
       'Positive' Class : Y               
                                          

In [ ]:
# ncol(mod_inp_mat)
# # sort(table(colnames(mod_inp_mat)), decreasing=TRUE)

# getSymbols("163059", 'org.Hs.eg.db')
# colnames(expression)[grep("ZNF433", colnames(expression))]